In [9]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import shutil
import subprocess

class SMT_test:
    chart = None
    def __init__(self, benchmarks_path='/home/hanrui/', chart_path='../charts', result_path='../results', log_path='..'):
        self.benchmarks_path = benchmarks_path
        self.chart_path = chart_path
        self.result_path = result_path
        self.log_path = log_path

    def init_chart(self, folder_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_folder_path = f'{self.benchmarks_path}{folder_path}'
        self.get_info(full_folder_path)
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)
    
    def get_info(self, folder_path):
        benchmarks = os.listdir(folder_path)
        for file in benchmarks:
            path = os.path.join(folder_path, file)
            if os.path.isdir(path):
                self.get_info(path)
            else:
                try:
                    with open(path) as f:
                        vars = 0
                        assertions = 0
                        status = 'none'
                        line = f.readline()
                        while line:
                            if line[:3] == '(de':
                                vars += 1
                            elif line[:3] == '(as':
                                assertions += 1
                            elif line[11:17] == 'status':
                                status = line.split(' ')[2][:-2]
                            line = f.readline()
                        category = path.split('/')[-2]
                        path = path.replace(self.benchmarks_path, '')
                        self.chart.loc[len(self.chart.index)] = [path, category, vars, assertions, status]
                except:
                    print(path)

    def move_files_to_folder(self, path_list, destination_folder):
        cnt = 0
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)
        else:
            shutil.rmtree(destination_folder)
            os.makedirs(destination_folder)
        for path in path_list:
            path = path.rstrip()
            file_name = os.path.basename(path)
            class_name = path.split('/')[4]
            destination_path = f"{destination_folder}/{class_name}/{file_name}"
            if os.path.isfile(path):
                if not os.path.exists(f"{destination_folder}/{class_name}"):
                    os.makedirs(f"{destination_folder}/{class_name}")
                shutil.copy(path, destination_path)
            elif not os.path.isfile(path):
                cnt += 1
                print(f'{path} is not a file')
        print(f"count: {cnt}")

    def get_need_path(self, solver, log_path, save_path):
        full_res_path = f'{self.result_path}/{solver}/{log_path}'
        results = os.listdir(full_res_path)
        count = 0
        file_path_list = []
        for file in tqdm(results, desc=full_res_path):
            with open(os.path.join(full_res_path, file)) as f:
                try:
                    while f.readline():
                        info = f.readline().rstrip()
                        result = info
                        if len(info) > 10:
                            result = "unknown"
                            info = f.readline().rstrip()
                        else:
                            f.readline()
                            info = f.readline().rstrip()
                        if result == 'unknown':
                            file_path_list.append(f"{self.benchmarks_path}{info.split(' : ')[0]}\n")
                            count += 1
                except:
                    print(file)
        self.move_files_to_folder(file_path_list, save_path)
        # print(file_path_list)
        # with open(f'{save_path}/count.txt', 'w') as f:
        #     for path in file_path_list:
        #         f.write(path + '\n')

        print(f"count: {count}")

    def get_unknown_path(self, solver, log_path, num_groups, prefix):
        full_file_path = f'{self.log_path}/{log_path}_unknown.log'
        tmp_file = open(full_file_path, "w")
        full_res_path = f'{self.result_path}/{solver}/{log_path}'
        results = os.listdir(full_res_path)
        count = 0
        for file in tqdm(results, desc=full_res_path):
            with open(os.path.join(full_res_path, file)) as f:
                try:
                    while f.readline():
                        info = f.readline().rstrip()
                        result = info
                        if len(info) > 10:
                            result = "unknown"
                            info = f.readline().rstrip()
                        else:
                            info = f.readline().rstrip()
                        if result == 'unknown':
                            tmp_file.write(f"{prefix}{info.split(' : ')[0]}\n")
                            count += 1
                except:
                    print(file)
        print(f"count: {count}")
        tmp_file.close()
        
        output_directory = f"{self.log_path}/{log_path}_unknown"
        if not os.path.exists(output_directory):
            os.mkdir(output_directory)
        else:
            shutil.rmtree(output_directory)
            os.mkdir(output_directory)
        shell_command = f'split -l $((($(wc -l < {full_file_path}) + {num_groups} - 1) / {num_groups})) {full_file_path} "{output_directory}/group_"'
        try:
            subprocess.run(shell_command, shell=True, check=True)
            print("Shell命令执行成功")
        except subprocess.CalledProcessError as e:
            print(f"Shell命令执行失败: {e}")

In [10]:
slia = SMT_test()
# slia.get_unknown_path('cvc5', 'QF_SLIA_unknown', 40, '/home/hanrui/')
slia.get_need_path('cvc5', 'QF_SLIA', '/home/hanrui/QF_SLIA_unknown')

../results/cvc5/QF_SLIA:   0%|          | 0/39 [00:00<?, ?it/s]

count: 0
count: 427
